# Computing distances for graph neural networks

## Part 1 - data preparation

Imports

In [75]:
import math
import os
import numpy as np
import pandas as pd
from pathlib import Path
from random import sample
from typing import Tuple
from geopy.distance import geodesic
from glob import glob
from sklearn.preprocessing import normalize
from enum import Enum
from sklearn.linear_model import LinearRegression
import requests


Constants needed for data reading

In [76]:
DATA_FOLDER = "Data"
GRAPH_INFO_TXT = "d07_text_meta_2021_03_27.txt"
current_directory = os.getcwd()
path_raw_data = os.path.join(current_directory, DATA_FOLDER)
path_raw_data_graph_info_txt = os.path.join(path_raw_data, GRAPH_INFO_TXT)


Set total number of nodes (nb_days) from Metadata (may contain empty nodes).

In [77]:
def DataReader_get_number_of_nodes(path_raw_data_graph_info_txt):
    nodes_location = []
    skip = True
    with open(path_raw_data_graph_info_txt) as f:
        content = f.readlines()
        for line in content:
            if skip:
                skip = False
            else:
                line = line.split('\t')
                line = line[:-1]         # ID     #LAT     #LONG
                nodes_location.append([line[0], line[8], line[9]])

    return len(nodes_location)


In [78]:
total_num_nodes = DataReader_get_number_of_nodes(path_raw_data_graph_info_txt)
print(f'The total number of nodes are : {total_num_nodes}')

The total number of nodes are : 4904


Get nodes that have data as good_nodes and nodes that have empty data as empty_nodes from the data.

In [79]:
def DataReader_get_good_empty_nodes(path_raw_data,total_num_nodes):
    index = total_num_nodes
    empty_nodes = []
    good_nodes = []
    txtFiles = os.path.join(path_raw_data, "*", "*.txt")
    for file in glob(txtFiles):
        with open(file) as f:
            content = f.readlines()
            for line in content:
                line = line.split(',')
                line = [line1.replace("\n", "") for line1 in line]
                if not (line[9] == '' or line[10] == '' or line[11] == ''):
                    good_nodes.append((int)(line[1]))
                else:
                    empty_nodes.append((int)(line[1]))
                index -= 1
                if index == 0:
                    return (good_nodes,empty_nodes)

In [80]:
(good_nodes, empty_nodes) = DataReader_get_good_empty_nodes(
    path_raw_data, total_num_nodes)
print(f'Five examples of nodes that contain data: {good_nodes[:5]}')
print(f'Five examples of nodes that do not have data: {empty_nodes[:5]}')


Five examples of nodes that contain data: [715898, 715918, 715920, 715929, 715930]
Five examples of nodes that do not have data: [715900, 715901, 715903, 715904, 715905]


Set data and labels as X and Y variables from Data. They will contain good nodes and empty nodes as well

In [81]:
def DataReader_read_data(path_raw_data):
    X = []
    Y = []
    txtFiles = os.path.join(path_raw_data, "*", "*.txt")
    nb_days = 0
    for file in glob(txtFiles):
        with open(file) as f:
            print(f'Reading day {nb_days + 1}')
            content = f.readlines()
            for line in content:
                line = line.split(',')
                line = [line1.replace("\n", "") for line1 in line]
                if not (line[9] == '' or line[10] == '' or line[11] == ''):
                    Y.append((float)(line[11]))
                    X.append([(float)(line[9]), (float)(line[10])])
        nb_days += 1
        # TODO : code for debugging, delete when finished
        # ------------------------------------------------
        if nb_days == 2:
            break
        # ------------------------------------------------
    X = normalize(np.array(X))
    Y = Y
    return X,Y,nb_days

In [82]:
X_data,Y_data,nb_days = DataReader_read_data(path_raw_data)

Reading day 1
Reading day 2


In [83]:
print(f'Total number of days read: {nb_days}')
print(f'Five examples of normalized input data: {X_data[:5]}')
print(f'Five examples of input data label: {Y_data[:5]}')

Total number of days read: 2
Five examples of normalized input data: [[9.99999984e-01 1.76732671e-04]
 [9.99999978e-01 2.08088231e-04]
 [9.99999974e-01 2.29906536e-04]
 [9.99999979e-01 2.03723400e-04]
 [9.99999976e-01 2.17045449e-04]]
Five examples of input data label: [70.4, 69.2, 66.0, 72.5, 69.8]


Get nodes geo location from Metadata. They will contain good nodes only

In [84]:
def DataReader_read_nodes_data(path_raw_data_graph_info_txt, good_nodes):

    nodes_location = []
    skip = True
    with open(path_raw_data_graph_info_txt) as f:
        content = f.readlines()
        for line in content:
            if skip:
                skip = False
            else:
                line = line.split('\t')
                line = line[:-1]
                if (int)(line[0]) in good_nodes:  # ID  #LAT    #LONG
                    nodes_location.append([line[0], line[8], line[9]])
    return nodes_location


In [85]:
nodes_location = DataReader_read_nodes_data(
    path_raw_data_graph_info_txt, good_nodes)
print(
    f'Five examples of nodes geolocation as well as their ID\'s {nodes_location[:5]}')

Five examples of nodes geolocation as well as their ID's [['715898', '33.880183', '-118.021787'], ['715918', '33.93311', '-118.091005'], ['715920', '33.938544', '-118.094941'], ['715929', '33.971707', '-118.123095'], ['715930', '33.971763', '-118.122905']]


Enumartion used for Dataset sizes

In [86]:
class Dataset(Enum):
    Experimental = 0
    ExperimentalManual = 1
    ExperimentalLR = 2
    Tiny = 3
    TinyManual = 4
    TinyLR = 5


In [87]:
class DatasetSize(Enum):
    Experimental = 8
    Tiny = 50

Function to set the nodes for the datasets such that they are the same throughout the project

More constants defined, as well as the nodes id's for the Experimental and Tiny dataset 

In [88]:
def Graph_get_nodes_for_dataset(dataset):
    experimental_all = [Dataset.Experimental,
                        Dataset.ExperimentalManual, Dataset.ExperimentalLR]
    tiny_all = [Dataset.Tiny, Dataset.TinyManual, Dataset.TinyLR]
    nodes_Experimental: list = [
        718292, 769496, 718291, 718290, 764567, 774279, 774278, 764671
    ]
    nodes_Tiny: list = [
        775637, 718165, 776986, 759289, 774672, 760643, 774671, 717046, 718419,
        769105, 764026, 759280, 775636, 759385, 760635, 718166, 774685, 774658,
        716938, 776177, 763453, 718421, 717045, 768598, 717043, 716063, 717041,
        717040, 717039, 737184, 717042, 718335, 763458, 776981, 737158, 737313,
        769118, 772501, 718173, 764037, 763447, 763246, 718041, 763251, 763424,
        763429, 763434, 763439, 764032, 764418
    ]
    if dataset in experimental_all:
        return nodes_Experimental
    elif dataset in tiny_all:
        return nodes_Tiny


In [89]:
print(Graph_get_nodes_for_dataset(Dataset.Experimental))


[718292, 769496, 718291, 718290, 764567, 774279, 774278, 764671]


In [90]:
def Graph_extract_time(json):
    try:
        return float(json['routes']['distance'])
    except KeyError:
        return 0


In [91]:
def Graph_OSRM_loop(p1: tuple, p2: tuple) -> float:
    requestUrl = f'http://router.project-osrm.org/route/v1/driving/{p1[1]},{p1[0]};{p2[1]},{p2[0]}'

    try:
        response = requests.get(requestUrl)
    except:
        return -1
    if (response.status_code != 204
            and response.headers["content-type"].strip().startswith(
                "application/json")):
        try:
            responseJson = response.json()
        except:
            return 0
    else:
        return -1
    if responseJson['code'] == "Ok":
        routes = responseJson['routes']
        routes.sort(key=Graph_extract_time, reverse=True)
        shortest_distance = float(routes[0]['distance']) * (1 / 1000)
        return shortest_distance
    return


In [92]:
def Graph_OSRM(p1: tuple, p2: tuple) -> float:
    distance = Graph_OSRM_loop(p1, p2)
    while (distance == -1):
        distance = Graph_OSRM_loop(p1, p2)
    return distance


In [93]:
p1 = ('33.880183', '-118.021787')
p2 = ('33.93311', '-118.091005')
print(
    f'Geodesic distance between node id 715898 and node id 715918 is {geodesic(p1,p2)}')
print(
    f'Road distance between node id 715898 and node id 715918 is {Graph_OSRM(p1,p2)} km')


Geodesic distance between node id 715898 and node id 715918 is 8.68599891018443 km
Road distance between node id 715898 and node id 715918 is 8.8494 km


In [94]:
def Graph_compute_all_OSRM_and_Geodesic(nodes_location, path_distances):
    for dataset in [Dataset.Experimental, Dataset.Tiny]:

        name_OSRM = os.path.join(
            path_distances, f'distances_OSRM_{dataset.name}.npy')
        name_geodesic = os.path.join(
            path_distances, f'distances_Geodesic_{dataset.name}.npy')

        if (os.path.exists(name_OSRM) and os.path.exists(name_geodesic)):
            print(f'Distances already computed for {dataset.name}')
            continue

        nodes_ids = Graph_get_nodes_for_dataset(dataset)
        nodes_location_dataset = [node for node in nodes_location if (int)(node[0]) in nodes_ids]

        matrix_size = len(nodes_location_dataset)

        OSRM_array = np.zeros((matrix_size, matrix_size))
        geodesic_array = np.zeros((matrix_size, matrix_size))

        for i in range(matrix_size - 1):
            for j in range(i + 1, matrix_size):
                p1 = ((float)(nodes_location_dataset[i][1]),
                      (float)(nodes_location_dataset[i][2]))
                p2 = ((float)(nodes_location_dataset[j][1]),
                      (float)(nodes_location_dataset[j][2]))

                id_1 = (int)(nodes_location_dataset[i][0])
                id_2 = (int)(nodes_location_dataset[j][0])

                print(f'Computing distances for Id\'s {id_1} and {id_2}')

                OSRM_array[i][j] = Graph_OSRM(p1, p2)
                geodesic_array[i][j] = geodesic(p1, p2).km
                OSRM_array[j][i] = Graph_OSRM(p2, p1)
                geodesic_array[j][i] = geodesic(p2, p1).km

        np.save(name_OSRM, OSRM_array)
        np.save(name_geodesic, geodesic_array)

    return


In [95]:
path_processed_data = os.path.join(current_directory, "Processed")
if not os.path.exists(path_processed_data):
    os.makedirs(path_processed_data)

path_distances = os.path.join(path_processed_data, "Distances")
if not os.path.exists(path_distances):
    os.makedirs(path_distances)


In [96]:
print("Started computing distances...")
Graph_compute_all_OSRM_and_Geodesic(nodes_location, path_distances)
print("Finished computing distances...")


Started computing distances...
Distances already computed for Experimental
Distances already computed for Tiny
Finished computing distances...


In [97]:
array_distances_OSRM_Experimental = np.load(os.path.join(
    path_distances, f'distances_OSRM_Experimental.npy'))
array_distances_Geodesic_Experimental = np.load(os.path.join(
    path_distances, f'distances_Geodesic_Experimental.npy'))
print(array_distances_OSRM_Experimental)
print(array_distances_Geodesic_Experimental)


[[0.     2.3406 3.5436 7.6358 7.1201 3.2997 3.6629 5.1336]
 [1.8412 0.     1.203  5.2952 4.7795 0.9591 1.3223 3.7805]
 [4.7693 7.1099 0.     4.0921 3.5765 8.069  6.1248 5.4476]
 [0.6772 3.0178 4.2208 0.     7.7973 3.9769 4.3401 5.8108]
 [1.1929 3.5335 0.7643 0.5157 0.     4.4926 2.5484 3.1371]
 [5.0133 7.3539 0.2439 4.3361 3.8204 0.     6.3688 5.6916]
 [3.573  5.9136 3.4803 5.0002 4.4845 6.8727 0.     2.4172]
 [3.1712 5.5117 2.5385 6.6306 6.1149 6.4709 3.2839 0.    ]]
[[0.         0.02477349 1.20259522 0.67630258 1.19130412 0.95912002
  1.15459812 1.1201794 ]
 [0.02477349 0.         1.20137815 0.67577782 1.19058306 0.95783532
  1.15965723 1.12529727]
 [1.20259522 1.20137815 0.         0.52659971 0.02639492 0.24354287
  0.30303407 0.30835885]
 [0.67630258 0.67577782 0.52659971 0.         0.51500154 0.28354955
  0.52112531 0.48918423]
 [1.19130412 1.19058306 0.02639492 0.51500154 0.         0.23375099
  0.27704067 0.28200812]
 [0.95912002 0.95783532 0.24354287 0.28354955 0.23375099 0.
  

In [98]:
class DistanceType(Enum):
    Geodesic = 0
    OSRM = 1


In [99]:
def Graph_get_adjency_matrix_weight(ID1_index, ID2_index, epsilon, sigma, distanceType,distances_array) -> float:
    distance = distances_array[ID1_index][ID2_index]
    weight = math.exp(-((distance**2) / (sigma**2)))
    if weight >= epsilon:
        return weight
    else:
        return 0
     

In [100]:
edge_index_Experimental_manual = [[0, 1, 7, 4, 7, 5], [1, 2, 4, 3, 6, 4]]
edge_index_Tiny_manual = [[
    0, 0, 0, 1, 5, 5, 5, 9, 9, 9, 10, 10, 10, 6, 14, 15, 7, 13, 10, 11, 8,
    9, 4, 16, 5, 2, 20, 3, 22, 23, 24, 25, 26, 28, 29, 30, 31, 21, 32, 33,
    34, 36, 17, 38, 12, 39, 40, 41, 42, 44, 45, 46, 47, 48, 27, 35, 9
],
    [
    1, 2, 3, 4, 6, 7, 2, 4, 3, 12, 12, 11,
    4, 14, 15, 12, 13, 10, 11, 8, 2, 3, 16,
    17, 6, 20, 21, 22, 23, 24, 25, 26, 27,
    29, 30, 31, 7, 32, 33, 34, 35, 9, 37,
    5, 39, 40, 41, 42, 43, 45, 46, 47, 48,
    0, 19, 18, 36
]]


In [101]:
def Graph_get_info_for_Standard(nodes_location, epsilon, sigma, dataset, distanceType,array_distances):
    nodes = Graph_get_nodes_for_dataset(dataset)
    edge_index = []
    edge_weight = []
    nodes_location_manual = [node for node in nodes_location if (int)(node[0]) in nodes]
    num_nodes = len(nodes_location_manual)
    for i in range(num_nodes - 1):
        for j in range(i, num_nodes - 1):
            if i != j:
                weight = Graph_get_adjency_matrix_weight(i,j,epsilon, sigma, distanceType,array_distances)
                if weight > 0:
                    edge_index.append([i, j])
                    edge_weight.append(weight)
    edge_index = np.transpose(edge_index)
    return edge_index, edge_weight

In [102]:
def Graph_get_info_for_Manual(nodes_location, epsilon, sigma, dataset, distanceType,
            edge_index_Experimental_manual,edge_index_Tiny_manual,array_distances):
    
    edge_weight = []
    if dataset == Dataset.ExperimentalManual:
        edge_index = edge_index_Experimental_manual
    elif dataset == Dataset.TinyManual:
        edge_index = edge_index_Tiny_manual
    nodes = Graph_get_nodes_for_dataset(dataset)
    nodes_location_standard = [node for node in nodes_location if (int)(node[0]) in nodes]
    num_nodes = len(nodes_location_standard)
    nodes_info = np.zeros((num_nodes, 3))
    for i in range(num_nodes):
        np.where(nodes == (int)(nodes_location_standard[i][0]))[0]
        nodes_info[np.where(nodes == (int)(nodes_location_standard[i][0]))[0]] = nodes_location_standard[i]
    nodes_location_standard = np.array(nodes_info)
    for i in range(len(edge_index[0])):
        weight = Graph_get_adjency_matrix_weight(
            edge_index[0][i],edge_index[1][i], epsilon, sigma, distanceType,array_distances
        )
        edge_weight.append(weight)
    return edge_index, edge_weight


In [105]:
def Graph_get_top_3_nodes_for_node_with_LR(node, dataset, nodes_location, speed_vector):
    nodes_ids = Graph_get_nodes_for_dataset(dataset)
    ids_index = 0
    X_train = []
    Y_train = []
    nodes_used = []
    node_ids_order = [(int)(node[0]) for node in nodes_location]
    nodes_used_computed = False
    X_train_snapshot = []
    for speed in speed_vector:

        if node != node_ids_order[ids_index] and node_ids_order[ids_index] in nodes_ids:
            X_train_snapshot.append(speed)
            if not nodes_used_computed:
                nodes_used.append(node_ids_order[ids_index])

        if node == node_ids_order[ids_index]:
            Y_train.append(speed)

        ids_index += 1
        if ids_index == len(node_ids_order): 
            ids_index = 0
            X_train.append(X_train_snapshot)
            X_train_snapshot = []
            nodes_used_computed = True

    regression = LinearRegression(positive=True).fit(X_train, Y_train)
    coeffiecients = regression.coef_.tolist()
    results = zip(nodes_used, coeffiecients)
    sorted_results = sorted(results, key=lambda tup: tup[1], reverse=True)
    best = sorted_results[:3]
    return [result[0] for result in best]

In [106]:
print(Graph_get_top_3_nodes_for_node_with_LR(717042, Dataset.TinyLR, nodes_location, Y_data))

[759385, 716938, 763246]


In [108]:
def Graph_get_info_for_LR(nodes_location, epsilon, sigma, dataset, distanceType,array_distances, Y_data):
    edge_index = []
    edge_weight = []
    nodes_ids = Graph_get_nodes_for_dataset(dataset)
    for node in nodes_ids:
        nodes_relevant = Graph_get_top_3_nodes_for_node_with_LR(node, dataset, nodes_location, Y_data)
        for node_relevant in nodes_relevant:
            edge_index.append([nodes_ids.index(node_relevant),nodes_ids.index(node)])
            edge_weight.append(Graph_get_adjency_matrix_weight(
                    nodes_ids.index(node_relevant), nodes_ids.index(node),
                    epsilon, sigma, distanceType,array_distances))
    edge_index = [list(x) for x in set(tuple(x) for x in edge_index)]
    edge_index = np.transpose(edge_index)
    return edge_index, edge_weight

In [110]:
def Graph_save_graph(nodes_location, epsilon, sigma, dataset, distanceType, path_processed_data,
                    edge_index_Experimental_manual,edge_index_Tiny_manual,path_distances,Y_data):

        nodes = Graph_get_nodes_for_dataset(dataset)
        name_folder_weight = os.path.join(path_processed_data,'EdgeWeight')
        name_folder_index = os.path.join(path_processed_data,'EdgeIndex')
        if dataset in [Dataset.Experimental,Dataset.ExperimentalLR,Dataset.ExperimentalManual]:
            dataset_name = Dataset.Experimental.name
        if dataset in [Dataset.Tiny,Dataset.TinyLR,Dataset.TinyManual]:
            dataset_name = Dataset.Tiny.name
        array_distances = np.load(os.path.join(path_distances,f'distances_{distanceType.name}_{dataset_name}.npy'))

        if not os.path.exists(name_folder_weight):
            os.makedirs(name_folder_weight)

        if not os.path.exists(name_folder_index):
            os.makedirs(name_folder_index)

        postfix = f'{distanceType.name}_{epsilon}_{sigma}_{dataset.name}'

        name_weight = os.path.join(name_folder_weight,f'weight_{postfix}.npy')
        name_index = os.path.join(name_folder_index,f'index_{postfix}.npy')

        if os.path.exists(name_weight) and os.path.exists(name_index): 
            print(f'Graph already saved with configuration : epsilon = {epsilon}, sigma = {sigma}, size = {dataset.name}, distance = {distanceType.name}')
            return

        print(f'Saving graph with configuration : epsilon = {epsilon}, sigma = {sigma}, size = {dataset.name}, distance = {distanceType.name}')
        if (dataset == Dataset.ExperimentalManual or dataset == Dataset.TinyManual):
            edge_index, edge_weight = Graph_get_info_for_Manual(nodes_location, epsilon, sigma, dataset, distanceType,
                    edge_index_Experimental_manual,edge_index_Tiny_manual,array_distances)
        elif (dataset == Dataset.ExperimentalLR or dataset == Dataset.TinyLR):
            edge_index, edge_weight = Graph_get_info_for_LR(nodes_location, epsilon, sigma, dataset, distanceType,array_distances,Y_data)
        else:
            edge_index, edge_weight = Graph_get_info_for_Standard(nodes_location, epsilon, sigma, dataset, distanceType,array_distances)
        
        np.save(name_index, edge_index)
        np.save(name_weight, edge_weight)

In [111]:
EPSILON_ARRAY = [0.1, 0.3, 0.5, 0.7]
SIGMA_ARRAY = [1, 3, 5, 10]


In [112]:
for epsilon in EPSILON_ARRAY:
    for sigma in SIGMA_ARRAY:
        for distanceType in DistanceType:
            for dataset in Dataset:
                Graph_save_graph(nodes_location, epsilon, sigma, dataset, distanceType, path_processed_data,
                    edge_index_Experimental_manual,edge_index_Tiny_manual,path_distances,Y_data)

Saving graph with configuration : epsilon = 0.1, sigma = 1, size = Experimental, distance = Geodesic
Saving graph with configuration : epsilon = 0.1, sigma = 1, size = ExperimentalManual, distance = Geodesic
Saving graph with configuration : epsilon = 0.1, sigma = 1, size = ExperimentalLR, distance = Geodesic
Saving graph with configuration : epsilon = 0.1, sigma = 1, size = Tiny, distance = Geodesic
Saving graph with configuration : epsilon = 0.1, sigma = 1, size = TinyManual, distance = Geodesic
Saving graph with configuration : epsilon = 0.1, sigma = 1, size = TinyLR, distance = Geodesic
Saving graph with configuration : epsilon = 0.1, sigma = 1, size = Experimental, distance = OSRM
Saving graph with configuration : epsilon = 0.1, sigma = 1, size = ExperimentalManual, distance = OSRM
Saving graph with configuration : epsilon = 0.1, sigma = 1, size = ExperimentalLR, distance = OSRM
Saving graph with configuration : epsilon = 0.1, sigma = 1, size = Tiny, distance = OSRM
Saving graph w

In [ ]:
def save_dataset():
    return

Enumeration used for the folders name in which the processed data will be saved

In [42]:
class FoldersProcesedNames(Enum):
    STCONV = 0
    LSTM = 1
    EdgeWeight = 2
    EdgeIndex = 3
    LinearRegression = 4

Function which will return true if the folders created after data processing exists

In [44]:
def Datareader_is_data_read(path_processed_data : str,folders_procesed_names : FoldersProcesedNames):
    for folder_name in folders_procesed_names:
        is_read_folder = os.path.isdir(os.path.join(path_processed_data, str(folder_name.name)))
        if not is_read_folder:
            return False
    return True

In [46]:
print(f'Data is read and stored: {Datareader_is_data_read(path_processed_data,FoldersProcesedNames)}')

Data is read and stored: False
